# Design Critique Strategy

**Mental Model:** Structured feedback process to refine and improve outputs

**Date:** [Fill in]  
**Author:** Leif Haven Martinson  

## Concept

In design and creative fields, critique is essential:
- Initial draft → Peer feedback → Revision
- Multiple perspectives identify blind spots
- Structured critique improves output quality

This notebook simulates a design crit session where:
1. Generate initial solution
2. Multiple agents provide structured feedback
3. Refine based on critique
4. (Optionally) Iterate

## How It Works

1. **Initial Draft** - Generate first version
2. **Critique Round** - Agents evaluate on different dimensions
3. **Synthesis** - Integrate feedback
4. **Revision** - Improve the draft
5. **Iterate** (optional) - Repeat critique/revision

## Use Cases

- Creative writing (stories, articles, copy)
- Technical writing (documentation, proposals)
- Code review and improvement
- Design reviews (UI, architecture, etc.)
- Research paper revision

In [ ]:
# Setup
import sys
sys.path.append('../code')

from harness import (
    llm_call,
    llm_call_stream,
    ExperimentConfig,
    ExperimentResult,
    get_tracker
)
from harness.defaults import DEFAULT_MODEL, DEFAULT_PROVIDER

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

print("✅ Setup complete")

## Configuration

**CUSTOMIZE HERE:** Define your critique panel

In [ ]:
# ========================================
# CRITIQUE CONFIGURATION - EDIT THESE
# ========================================

# Model configuration
PROVIDER = DEFAULT_PROVIDER
MODEL = DEFAULT_MODEL
TEMPERATURE = 0.7

# Number of critique/revision iterations
NUM_ITERATIONS = 2  # 🔧 1 = single critique, 2+ = iterative refinement

# ========================================
# DEFINE YOUR CRITIQUE PANEL
# ========================================
# Each critic evaluates on specific dimensions

CRITIQUE_PANEL = [
    {
        "name": "Clarity Critic",
        "focus": "Clarity and comprehension",
        "criteria": """Evaluate clarity and comprehension:
- Is the writing clear and easy to understand?
- Are concepts explained well?
- Is the structure logical?
- Are there any confusing or ambiguous parts?

Provide specific suggestions for improving clarity."""
    },
    {
        "name": "Technical Accuracy Critic",
        "focus": "Technical correctness and accuracy",
        "criteria": """Evaluate technical accuracy:
- Are facts and claims accurate?
- Are technical details correct?
- Are there any errors or misconceptions?
- Are sources and evidence appropriate?

Point out any inaccuracies and suggest corrections."""
    },
    {
        "name": "Engagement Critic",
        "focus": "Reader engagement and interest",
        "criteria": """Evaluate engagement:
- Is it interesting and engaging to read?
- Does it hook the reader?
- Is the tone appropriate?
- Are examples compelling?

Suggest ways to make it more engaging."""
    },
    {
        "name": "Completeness Critic",
        "focus": "Completeness and coverage",
        "criteria": """Evaluate completeness:
- Are all important points covered?
- Are there gaps or missing information?
- Is enough context provided?
- Are counterarguments addressed?

Identify what's missing and should be added."""
    },
    {
        "name": "Style Critic",
        "focus": "Writing style and quality",
        "criteria": """Evaluate writing style:
- Is the writing polished and professional?
- Is word choice appropriate?
- Are sentences well-constructed?
- Is there good variety in sentence structure?

Suggest stylistic improvements."""
    },
    
    # Add custom critics as needed:
    # {
    #     "name": "Audience Critic",
    #     "focus": "Audience appropriateness",
    #     "criteria": "Consider the target audience..."
    # },
]

# ========================================
# REVISION PROMPT
# ========================================
# How revisions incorporate feedback

REVISION_PROMPT = """You are revising a draft based on structured critique.

Your job:
1. Carefully review all critiques
2. Identify the most important improvements
3. Revise the draft to address feedback
4. Balance different critique perspectives
5. Maintain the core message while improving quality

Provide an improved version that addresses the feedback."""

print(f"✅ Critique panel configured:")
print(f"   - {len(CRITIQUE_PANEL)} critics")
print(f"   - Iterations: {NUM_ITERATIONS}")
print(f"\n🎨 Your critique panel:")
for critic in CRITIQUE_PANEL:
    print(f"   - {critic['name']}: {critic['focus']}")

## Define Task

**CUSTOMIZE HERE:** What should be created and critiqued?

In [ ]:
# ========================================
# TASK DEFINITION - EDIT THIS
# ========================================

TASK_PROMPT = """Write a 300-word blog post explaining what multi-agent AI systems are and why they matter.

Target audience: Technical professionals who are curious about AI but not AI experts

Goals:
- Explain the concept clearly
- Provide concrete examples
- Make it engaging and accessible
- End with why this matters for the future
"""

# Alternative tasks to try:
#
# "Write API documentation for a new authentication endpoint."
#
# "Draft a product announcement email for our new feature launch."
#
# "Create a technical proposal for migrating our database to PostgreSQL."
#
# "Write a compelling product description for an AI-powered writing assistant."

print("📝 Task defined:")
print(TASK_PROMPT[:200] + "...")

## Generate Initial Draft

In [ ]:
import time

# Track timing and tokens
start_time = time.time()
total_tokens_in = 0
total_tokens_out = 0

print(f"\n{'='*80}")
print(f"📄 GENERATING INITIAL DRAFT")
print(f"{'='*80}\n")

# Generate initial draft (streaming for visibility)
full_text = ""
response = None

for chunk in llm_call_stream(TASK_PROMPT, provider=PROVIDER, model=MODEL, temperature=TEMPERATURE):
    if isinstance(chunk, str):
        print(chunk, end="", flush=True)
        full_text += chunk
    else:
        response = chunk

print("\n")

current_draft = response.text
total_tokens_in += response.tokens_in or 0
total_tokens_out += response.tokens_out or 0

print(f"✅ Initial draft generated ({len(current_draft)} characters)")

# Store versions for comparison
versions = [{
    "version": 0,
    "label": "Initial Draft",
    "content": current_draft
}]

## Critique & Revision Iterations

In [ ]:
for iteration in range(NUM_ITERATIONS):
    print(f"\n{'='*80}")
    print(f"🎨 ITERATION {iteration + 1}: Critique & Revision")
    print(f"{'='*80}\n")
    
    # === CRITIQUE PHASE ===
    print(f"\n{'─'*80}")
    print(f"💬 CRITIQUE PHASE")
    print(f"{'─'*80}\n")
    
    critiques = []
    
    for critic in CRITIQUE_PANEL:
        print(f"\n{critic['name']}:")
        print("-" * 60)
        
        # Create critique prompt
        critique_prompt = f"""You are providing critique as: {critic['name']}

{critic['criteria']}

Draft to critique:
\"\"\"
{current_draft}
\"\"\"

Provide your critique focusing on {critic['focus']}:"""
        
        # Get critique (streaming)
        full_text = ""
        response = None
        
        for chunk in llm_call_stream(critique_prompt, provider=PROVIDER, model=MODEL, temperature=0.7):
            if isinstance(chunk, str):
                print(chunk, end="", flush=True)
                full_text += chunk
            else:
                response = chunk
        
        print("\n")
        
        critiques.append({
            "critic": critic['name'],
            "focus": critic['focus'],
            "feedback": response.text
        })
        
        total_tokens_in += response.tokens_in or 0
        total_tokens_out += response.tokens_out or 0
    
    # === REVISION PHASE ===
    print(f"\n{'─'*80}")
    print(f"✏️  REVISION PHASE")
    print(f"{'─'*80}\n")
    
    # Build revision prompt
    revision_prompt = f"{REVISION_PROMPT}\n\n"
    revision_prompt += f"Current Draft:\n\"\"\"{current_draft}\"\"\"\n\n"
    revision_prompt += "Critiques:\n\n"
    
    for critique in critiques:
        revision_prompt += f"{critique['critic']} ({critique['focus']}):\n"
        revision_prompt += f"{critique['feedback']}\n\n"
        revision_prompt += "-" * 60 + "\n\n"
    
    revision_prompt += "Revised draft (addressing the critiques):"
    
    # Get revision (streaming)
    full_text = ""
    response = None
    
    for chunk in llm_call_stream(revision_prompt, provider=PROVIDER, model=MODEL, temperature=0.7):
        if isinstance(chunk, str):
            print(chunk, end="", flush=True)
            full_text += chunk
        else:
            response = chunk
    
    print("\n")
    
    current_draft = response.text
    total_tokens_in += response.tokens_in or 0
    total_tokens_out += response.tokens_out or 0
    
    # Store version
    versions.append({
        "version": iteration + 1,
        "label": f"After Iteration {iteration + 1}",
        "content": current_draft,
        "critiques": critiques
    })
    
    print(f"✅ Iteration {iteration + 1} complete")

# Calculate total time
total_time = time.time() - start_time

print(f"\n{'='*80}")
print(f"✅ CRITIQUE PROCESS COMPLETE")
print(f"{'='*80}")
print(f"\n📊 Stats:")
print(f"   Critics: {len(CRITIQUE_PANEL)}")
print(f"   Iterations: {NUM_ITERATIONS}")
print(f"   Versions: {len(versions)}")
print(f"   Total time: {total_time:.1f}s")
print(f"   Total tokens: {total_tokens_in + total_tokens_out:,}")

## Compare Versions

In [ ]:
print(f"\n{'='*80}")
print(f"📊 VERSION COMPARISON")
print(f"{'='*80}\n")

for version in versions:
    print(f"\n{'─'*80}")
    print(f"Version {version['version']}: {version['label']}")
    print(f"{'─'*80}\n")
    print(version['content'])
    print(f"\n(Length: {len(version['content'])} characters)")

print(f"\n{'='*80}")

## Final Output

In [ ]:
print(f"\n{'='*80}")
print(f"📄 FINAL OUTPUT")
print(f"{'='*80}\n")
print(current_draft)
print(f"\n{'='*80}")

## Quality Analysis

Measure improvement across iterations

In [ ]:
# Use LLM to evaluate quality progression
from harness import llm_call

print(f"\n📊 Evaluating quality progression...\n")

comparison_prompt = f"""Compare these versions of the same content and rate how much the quality improved.

Original (Version 0):
\"\"\"
{versions[0]['content']}
\"\"\"

Final (Version {len(versions)-1}):
\"\"\"
{versions[-1]['content']}
\"\"\"

Rate the improvement on a scale of 1-10 for each dimension:
- Clarity
- Technical Accuracy
- Engagement
- Completeness
- Writing Style

Also provide an overall quality improvement score (1-10) and brief analysis.

Format:
Clarity: X/10
Technical Accuracy: X/10
Engagement: X/10
Completeness: X/10
Writing Style: X/10
Overall Improvement: X/10

Analysis: [Brief explanation of key improvements]
"""

evaluation = llm_call(comparison_prompt, provider=PROVIDER, model=MODEL, temperature=0.3)

print(evaluation.text)

## Save Results

In [ ]:
# Save to file
import json
from datetime import datetime
from pathlib import Path

# Create results directory
results_dir = Path("../experiments/design_critique")
results_dir.mkdir(parents=True, exist_ok=True)

# Create result object
result = {
    "task": TASK_PROMPT,
    "critics": [{"name": c["name"], "focus": c["focus"]} for c in CRITIQUE_PANEL],
    "versions": versions,
    "final_output": current_draft,
    "num_iterations": NUM_ITERATIONS,
    "stats": {
        "total_time_s": total_time,
        "total_tokens": total_tokens_in + total_tokens_out,
        "model": MODEL,
        "provider": PROVIDER
    },
    "timestamp": datetime.now().isoformat()
}

# Save JSON
filename = results_dir / f"result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w') as f:
    json.dump(result, f, indent=2)

print(f"\n💾 Saved to: {filename}")

## Next Steps

### Try Different Configurations

1. **Different tasks** - Try various writing types (technical, creative, business)
2. **More iterations** - See if quality keeps improving (diminishing returns?)
3. **Different critics** - Add domain-specific critics for your use case
4. **Fewer critics** - Test if 2-3 focused critics work as well as 5
5. **Compare to single pass** - Does critique actually help?

### Custom Critique Panels

**For code review:**
- Functionality critic
- Performance critic
- Security critic
- Maintainability critic

**For academic writing:**
- Rigor critic (methodology)
- Clarity critic (communication)
- Novelty critic (contribution)
- References critic (citations)

**For marketing copy:**
- Brand voice critic
- Conversion critic
- SEO critic
- Emotional appeal critic